In [1]:
import os
import torch
from transformers import AutoModel, AutoTokenizer
import torch.nn.functional as F

In [2]:
torch.manual_seed(42)

if torch.cuda.is_available():
    device="cuda"
    print(f"[INFO] GPU disponible. Usaremos CUDA y limitaremos a ~3800 MiB de VRAM.")
else:
    device="cpu"
    print("[WARN] GPU no disponible. Usando CPU sin limitación de memoria.")

[INFO] GPU disponible. Usaremos CUDA y limitaremos a ~3800 MiB de VRAM.


In [ ]:
model=AutoModel.from_pretrained(
    "../models/jina-reranker-m0",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    max_memory={
        0: "3500MB"
    }
)
model.eval()

tokenizer=AutoTokenizer.from_pretrained(
    "../models/jina-reranker-m0",
    trust_remote_code=True
)

Some parameters are on the meta device because they were offloaded to the disk.


In [4]:
table_reports="""
CREATE TABLE INVENTARIO_REPORTES (
    ID_INVENTARIO_REPORTE int NOT NULL,
    CLAVE_REP varchar(100) NOT NULL,
    CLAVE_PAIS varchar(10) NULL,
    CLAVE_ENTIDADREGULADA varchar(10) NULL,
    CLAVE_REG varchar(10) NULL,
    CLAVE_SERIE varchar(15) NULL,
    SUBSERIE varchar(15) NULL,
    CLAVE_GRUPO varchar(15) NULL,
    REPORTE varchar(100) NULL,
    CLAVE_SECCION_REP varchar(50) NULL,
    CLAVE_VERSION_REPORTE varchar(3) NULL,
    CLAVE_PERIODO varchar(15) NULL,
    DESCRIPCION_ESP varchar(500) NULL,
    CLAVE_FECHA_ENT_REP varchar(100) NULL,
    CARACTERISTICAS varchar(500) NULL,
    CLAVE_REGULACION_REP varchar(50) NULL,
    FECHA_ALTA datetime NULL,
    FECHA_ACTUALIZADA datetime NULL,
    CLAVE_REP_GENERAL varchar(100) NULL,
    CLASIFICACION date NULL,
    VIGENTE INTEGER NULL,
    CONSTRAINT PK_CLAVE_REP PRIMARY KEY (CLAVE_REP)
)
"""

table_valid="""
CREATE TABLE dbo.INVENTARIO_VALIDACIONES (
    ID_INVENTARIO_VALIDACION int NOT NULL,
    CLAVE_VALIDACION varchar(100) NOT NULL,
    CLAVE_PAIS varchar(10) NULL,
    CLAVE_ENTIDADREGULADA varchar(10) NULL,
    CLAVE_REG varchar(10) NULL,
    CLAVE_REP varchar(100) NOT NULL,
    ID_VALIDACION_ANT varchar(100) NULL,
    DESCRIPCION_VALIDACION varchar(2000) NULL,
    TIPO_VALIDACION varchar(30) NULL,
    TIPO_VALIDACION_CALC varchar(50) NULL,
    FECHA_ALTA datetime NULL,
    FECHA_ACTUALIZADA datetime NULL,
    ID_VAL varchar(15) NULL,
    CAMPO varchar(250) NULL,
    CONSTRAINT PK_VALIDACION PRIMARY KEY CLUSTERED (CLAVE_VALIDACION)
)
"""

table_compuest="""
CREATE TABLE dbo.INVENTARIO_VALIDACIONES (
    ID_INVENTARIO_VALIDACION int NOT NULL,
    CLAVE_VALIDACION varchar(100) NOT NULL,
    CLAVE_PAIS varchar(10) NULL,
    CLAVE_ENTIDADREGULADA varchar(10) NULL,
    CLAVE_REG varchar(10) NULL,
    CLAVE_REP varchar(100) NOT NULL,
    ID_VALIDACION_ANT varchar(100) NULL,
    DESCRIPCION_VALIDACION varchar(2000) NULL,
    TIPO_VALIDACION varchar(30) NULL,
    TIPO_VALIDACION_CALC varchar(50) NULL,
    FECHA_ALTA datetime NULL,
    FECHA_ACTUALIZADA datetime NULL,
    ID_VAL varchar(15) NULL,
    CAMPO varchar(250) NULL,
    CONSTRAINT PK_VALIDACION PRIMARY KEY CLUSTERED (CLAVE_VALIDACION)
)

CREATE TABLE INVENTARIO_REPORTES (
    ID_INVENTARIO_REPORTE int NOT NULL,
    CLAVE_REP varchar(100) NOT NULL,
    CLAVE_PAIS varchar(10) NULL,
    CLAVE_ENTIDADREGULADA varchar(10) NULL,
    CLAVE_REG varchar(10) NULL,
    CLAVE_SERIE varchar(15) NULL,
    SUBSERIE varchar(15) NULL,
    CLAVE_GRUPO varchar(15) NULL,
    REPORTE varchar(100) NULL,
    CLAVE_SECCION_REP varchar(50) NULL,
    CLAVE_VERSION_REPORTE varchar(3) NULL,
    CLAVE_PERIODO varchar(15) NULL,
    DESCRIPCION_ESP varchar(500) NULL,
    CLAVE_FECHA_ENT_REP varchar(100) NULL,
    CARACTERISTICAS varchar(500) NULL,
    CLAVE_REGULACION_REP varchar(50) NULL,
    FECHA_ALTA datetime NULL,
    FECHA_ACTUALIZADA datetime NULL,
    CLAVE_REP_GENERAL varchar(100) NULL,
    CLASIFICACION date NULL,
    VIGENTE INTEGER NULL,
    CONSTRAINT PK_CLAVE_REP PRIMARY KEY (CLAVE_REP)
)
"""

table_client="""
CREATE TABLE dbo.CHI_CLIENTE (
    ID_CLIENTE_CHI INT NOT NULL,
    CLAVE_CLIENTE VARCHAR(50) NOT NULL,
    SOPORTE_SLA INT NULL,
    PMRV INT NULL,
    CCI INT NULL,
    P_ACTIVOS INT NULL,
    CARTERA INT NULL,
    ACT_PRODUCTO INT NULL,
    SUMA INT NULL,
    EDS INT NULL,
    CONSTRAINT PK_CHI_CLIENTE PRIMARY KEY (CLAVE_CLIENTE)
);
"""

table_docs={
    "INVENTARIO_REPORTES": table_reports,
    "INVENTARIO_VALIDACIONES": table_valid,
    "INVENTARIO_VALIDACIONES_COMPUESTO": table_compuest,
    "CHI_CLIENTE": table_client
}

In [5]:
def choose_table(query: str) -> str:
    pairs=[[query, schema] for schema in table_docs.values()]

    with torch.no_grad():
        scores=model.compute_score(pairs, max_length=1024, doc_type="text")

    best_idx=int(torch.argmax(torch.tensor(scores, device="cpu")))
    return list(table_docs.keys())[best_idx]

In [9]:
ejemplos=[
    "Qué reportes son de BM",
    "Cuántas validaciones son de tipo INTRA",
    "Mostrar validaciones automáticas",
    "Listar informes de casas de bolsa"
    "De los reportes de CB cuales tienen validaciones de tipo intra",
    "Cual es el estado de los activos de santander"
]
for q in ejemplos:
    tabla=choose_table(q)
    print(f"→ “{q}”  se refiere más a: {tabla}")

→ “Qué reportes son de BM”  se refiere más a: INVENTARIO_REPORTES
→ “Cuántas validaciones son de tipo INTRA”  se refiere más a: INVENTARIO_VALIDACIONES
→ “Mostrar validaciones automáticas”  se refiere más a: INVENTARIO_VALIDACIONES
→ “Listar informes de casas de bolsaDe los reportes de CB cuales tienen validaciones de tipo intra”  se refiere más a: INVENTARIO_VALIDACIONES_COMPUESTO
→ “Cual es el estado de los activos de santander”  se refiere más a: INVENTARIO_VALIDACIONES_COMPUESTO
